# Cleaning mock client data

For this notebook, I'll be showcasing some methods I use for cleaning messy data. While this dataset I created is not a standard financial report, it contains fields that I've found most likely to contain errors. Humans are prone to error, especially if the work is mundane and there are no data validation measures put in place.

In [186]:
import pandas as pd
import pandas_dedupe

original_data = pd.read_csv('C:/Users/Tyler/Messy_CSV.csv')
original_data.head()

,Client,Payment,Date_of_service,Location
0,Alpha,"$5,000",1/1/2022,Philadelphia
1,alpha,4000,"Saturday, January 15, 2022",Philedelphia
2,alpha,4000,2/1/2022,Philaelphia
3,Alfa,"$5,000",3/15/2021,Philadelphia
4,lpha,"$60,000,000.00",3/28/2022,Philadelphia


### .dedupe_dataframe() method
In the first section, I'll be cleaning string data with the pandas_dedupe module.
--For Windows users, we need to have Microsoft Visual C++ installed for the module to perform as it should--
the pandas_dedupe module allows us to use Machine Learning to identify duplicate records that may have different spellings, trailing white spaces, different capitalization, etc.
The .gazetteer_dataframe() method also allows for ML categorical matching to a canonical list, 'gazette', that contains the correct spelling for the strings in question.

I already trained this model, but if its your first time, you will be prompted to answer (y)es, (n)o, (u)ncertain, (f)inish, and (p)revious to train the model yourself. Massive sets may take some training. See the [documentation](https://pypi.org/project/pandas-dedupe/) for more on how this module works.

In [187]:
df = pandas_dedupe.dedupe_dataframe( 
    original_data, 
    ['Client', 'Location'],
    canonicalize=True,
    )
print(df)

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...


C:\Users\Tyler\anaconda3\lib\site-packages\pandas_dedupe\utility_functions.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('[^\w\s\.\-\(\)\,\:\/\\\\]','')


# duplicate sets 3
      Client        Payment             Date_of_service      Location  \
0      alpha          5,000                    1/1/2022  philadelphia   
1      alpha           4000  saturday, january 15, 2022  philedelphia   
2      alpha           4000                    2/1/2022   philaelphia   
3       alfa          5,000                   3/15/2021  philadelphia   
4       lpha  60,000,000.00                   3/28/2022  philadelphia   
5      alpha           5000                    4/1/2022  philadelphia   
6      betha           4000                    1/2/2022      new york   
7       beta            400                   1/16/2022      mew york   
8      betha           5000                    2/2/2022      new york   
9       beta          4,000                   2/16/2022       new yol   
10    charly           5000                       3-jan            la   
11   charley           5000                    3-feb-22          l.a.   
12   charles          50000     

Note that 'betha' was returned instead of 'beta'. Had I added more 'Beta' values to the original dataframe, the model would changed its canonical representation to the correct 'beta'. I decided to keep the incorrect 'betha' and use it to include some variation in my example for replacing values (charlie and los angeles) below.

In [188]:
df['canonical_Client'] = df['canonical_Client'].replace(to_replace=['betha', 'charley'], value=['beta', 'charlie'])
df['canonical_Location'] = df['canonical_Location'].replace(to_replace='la', value='los angeles')

print(df)

      Client        Payment             Date_of_service      Location  \
0      alpha          5,000                    1/1/2022  philadelphia   
1      alpha           4000  saturday, january 15, 2022  philedelphia   
2      alpha           4000                    2/1/2022   philaelphia   
3       alfa          5,000                   3/15/2021  philadelphia   
4       lpha  60,000,000.00                   3/28/2022  philadelphia   
5      alpha           5000                    4/1/2022  philadelphia   
6      betha           4000                    1/2/2022      new york   
7       beta            400                   1/16/2022      mew york   
8      betha           5000                    2/2/2022      new york   
9       beta          4,000                   2/16/2022       new yol   
10    charly           5000                       3-jan            la   
11   charley           5000                    3-feb-22          l.a.   
12   charles          50000                    3/5/

### Replacing the messy strings with the clean strings

In [189]:
original_data[['Client', 'Location']] = df[['canonical_Client', 'canonical_Location']]
print(original_data)

     Client          Payment             Date_of_service      Location
0     alpha          $5,000                     1/1/2022  philadelphia
1     alpha             4000  Saturday, January 15, 2022  philadelphia
2     alpha             4000                    2/1/2022  philadelphia
3     alpha          $5,000                    3/15/2021  philadelphia
4     alpha  $60,000,000.00                    3/28/2022  philadelphia
5     alpha             5000                    4/1/2022  philadelphia
6      beta             4000                    1/2/2022      new york
7      beta              400                   1/16/2022      new york
8      beta             5000                    2/2/2022      new york
9      beta          $4,000                    2/16/2022      new york
10  charlie             5000                       3-Jan   los angeles
11  charlie             5000                    3-Feb-22   los angeles
12  charlie            50000                    3/5/2022   los angeles
13  ch

### Alright, time to clean up some numbers
Here's a basic function that I use for cleaning unwanted characters from my data.

In [190]:
# List of characters to remove
chars_to_remove = ['.00', ',', '$', ' ']

for character in chars_to_remove:
    # Replace the character with an empty string
    original_data['Payment'] = original_data['Payment'].apply(lambda x: x.replace(character, ''))
        
original_data['Payment']

0         5000
1         4000
2         4000
3         5000
4     60000000
5         5000
6         4000
7          400
8         5000
9         4000
10        5000
11        5000
12       50000
13        5000
Name: Payment, dtype: object

In this fictional case, I imagine that all the payments are in the thousands. In the real world, that isn't so simple. Unfortunately, we may have to refer to a different table or pull up an invoice to find the correct value.

But for fun, I put some unnecessary constraints to see what I could do.

By transforming the data type into a string, I was able to quickly remove any 0s with regular expressions and then convert it back to integer and multiply by 1000. I really don't see when I would have to do this, but it was fun nonetheless. It also shows off the left to right processing of stacked methods/transformations.


In [191]:
original_data['Payment'] = original_data['Payment'].astype('str').str.replace(r'(0+)', '').astype('int')*1000
print(original_data)

     Client  Payment             Date_of_service      Location
0     alpha     5000                    1/1/2022  philadelphia
1     alpha     4000  Saturday, January 15, 2022  philadelphia
2     alpha     4000                    2/1/2022  philadelphia
3     alpha     5000                   3/15/2021  philadelphia
4     alpha     6000                   3/28/2022  philadelphia
5     alpha     5000                    4/1/2022  philadelphia
6      beta     4000                    1/2/2022      new york
7      beta     4000                   1/16/2022      new york
8      beta     5000                    2/2/2022      new york
9      beta     4000                   2/16/2022      new york
10  charlie     5000                       3-Jan   los angeles
11  charlie     5000                    3-Feb-22   los angeles
12  charlie     5000                    3/5/2022   los angeles
13  charlie     5000                   3/30/2022   los angeles


C:\Users\Tyler\AppData\Local\Temp/ipykernel_2644/1218524444.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  original_data['Payment'] = original_data['Payment'].astype('str').str.replace(r'(0+)', '').astype('int')*1000


### Now lets clean up some incorret date formats


In [192]:
original_data['Date_of_service'] = pd.to_datetime(original_data['Date_of_service'], errors = 'coerce')

print(original_data)

     Client  Payment Date_of_service      Location
0     alpha     5000      2022-01-01  philadelphia
1     alpha     4000      2022-01-15  philadelphia
2     alpha     4000      2022-02-01  philadelphia
3     alpha     5000      2021-03-15  philadelphia
4     alpha     6000      2022-03-28  philadelphia
5     alpha     5000      2022-04-01  philadelphia
6      beta     4000      2022-01-02      new york
7      beta     4000      2022-01-16      new york
8      beta     5000      2022-02-02      new york
9      beta     4000      2022-02-16      new york
10  charlie     5000             NaT   los angeles
11  charlie     5000      2022-02-03   los angeles
12  charlie     5000      2022-03-05   los angeles
13  charlie     5000      2022-03-30   los angeles


After having to coerce the .to_datetime function, we need to look for missing values in the Date_of_service column and fix them.

As we can see below, we only have one.

In [193]:
original_data[original_data['Date_of_service'].isna()]

,Client,Payment,Date_of_service,Location
10,charlie,5000,NaT,los angeles


We would probably need to reference an invoice or different table to find the date of service for this value.

In [195]:
original_data.loc[10,['Date_of_service']] = pd.to_datetime('2022-01-03')
print(original_data)

     Client  Payment Date_of_service      Location
0     alpha     5000      2022-01-01  philadelphia
1     alpha     4000      2022-01-15  philadelphia
2     alpha     4000      2022-02-01  philadelphia
3     alpha     5000      2021-03-15  philadelphia
4     alpha     6000      2022-03-28  philadelphia
5     alpha     5000      2022-04-01  philadelphia
6      beta     4000      2022-01-02      new york
7      beta     4000      2022-01-16      new york
8      beta     5000      2022-02-02      new york
9      beta     4000      2022-02-16      new york
10  charlie     5000      2022-01-03   los angeles
11  charlie     5000      2022-02-03   los angeles
12  charlie     5000      2022-03-05   los angeles
13  charlie     5000      2022-03-30   los angeles


Lets make sure that all the dates are in 2022.

We can clearly see there is one, but larger data might contain more errors (due to manual entry, loading error, transformation error, etc). To find if there are any dates that aren't within 2022, I'll first save a series of boolean values and then use a list comprehension to find the index for the dates outside of 2022.


In [196]:
mislabeled_years = (original_data['Date_of_service'] < '2022-01-01') | (original_data['Date_of_service'] > '2022-12-31')
mislabeled_dates_index = [i for i, x in enumerate(mislabeled_years) if x]
original_data.loc[mislabeled_dates_index, ['Date_of_service']] = pd.to_datetime('2022')
print(original_data)

     Client  Payment Date_of_service      Location
0     alpha     5000      2022-01-01  philadelphia
1     alpha     4000      2022-01-15  philadelphia
2     alpha     4000      2022-02-01  philadelphia
3     alpha     5000      2022-01-01  philadelphia
4     alpha     6000      2022-03-28  philadelphia
5     alpha     5000      2022-04-01  philadelphia
6      beta     4000      2022-01-02      new york
7      beta     4000      2022-01-16      new york
8      beta     5000      2022-02-02      new york
9      beta     4000      2022-02-16      new york
10  charlie     5000      2022-01-03   los angeles
11  charlie     5000      2022-02-03   los angeles
12  charlie     5000      2022-03-05   los angeles
13  charlie     5000      2022-03-30   los angeles


In [197]:
original_data['Client'] = original_data['Client'].str.capitalize() #Capitalizing the appropriate strings (if needed)
original_data['Location'] = original_data['Location'].str.title()

print(original_data)

     Client  Payment Date_of_service      Location
0     Alpha     5000      2022-01-01  Philadelphia
1     Alpha     4000      2022-01-15  Philadelphia
2     Alpha     4000      2022-02-01  Philadelphia
3     Alpha     5000      2022-01-01  Philadelphia
4     Alpha     6000      2022-03-28  Philadelphia
5     Alpha     5000      2022-04-01  Philadelphia
6      Beta     4000      2022-01-02      New York
7      Beta     4000      2022-01-16      New York
8      Beta     5000      2022-02-02      New York
9      Beta     4000      2022-02-16      New York
10  Charlie     5000      2022-01-03   Los Angeles
11  Charlie     5000      2022-02-03   Los Angeles
12  Charlie     5000      2022-03-05   Los Angeles
13  Charlie     5000      2022-03-30   Los Angeles


In [198]:
original_data.to_csv('Clean_CSV.csv') #Saving the file